In [25]:
from os import listdir
from os.path import isfile, join
import cv2
from matplotlib import pyplot as plt
import kerem
import numpy as np
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 15]

root_path = '../datasets/'
file_names = [f for f in listdir(root_path) if isfile(join(root_path, f))]

image_storage = dict()

for file_name in file_names:
    image_storage[file_name.split('.')[0]] = cv2.imread(root_path + file_name)

In [26]:
cv2.__version__

'4.5.2'

In [27]:

img1 = image_storage['mf01']
img2 = image_storage['wf']


#kerem.show_images([kerem.resize_image(img1,3), kerem.resize_image(img2,3)])
kerem.show_images([img1, img2])

In [28]:
surf = cv2.xfeatures2d.SURF_create(hessianThreshold=400, )
kp1, des1 = surf.detectAndCompute(img1,None)
kp2, des2 = surf.detectAndCompute(img2,None)

In [29]:
print(len(kp1))
print(len(kp2))

144557
24933


In [30]:
#-- Step 2: Matching descriptor vectors with a FLANN based matcher
# Since SURF is a floating-point descriptor NORM_L2 is used
matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
knn_matches = matcher.knnMatch(des1, des2, 2)


In [32]:
#-- Filter matches using the Lowe's ratio test
ratio_thresh = 0.6
good_matches = []
for m,n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)
len(good_matches)

11

In [33]:
#-- Draw matches
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
cv2.drawMatches(img1, kp1, img2, kp2, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

array([[[ 90,  82,  69],
        [ 92,  83,  70],
        [ 91,  82,  69],
        ...,
        [133, 121, 115],
        [126, 112, 106],
        [129, 115, 109]],

       [[ 93,  83,  73],
        [ 94,  84,  74],
        [ 95,  83,  71],
        ...,
        [137, 125, 121],
        [136, 122, 116],
        [132, 118, 112]],

       [[ 98,  86,  76],
        [ 98,  86,  76],
        [ 94,  85,  72],
        ...,
        [135, 122, 120],
        [134, 122, 116],
        [133, 121, 115]],

       ...,

       [[ 69,  69,  63],
        [ 72,  69,  64],
        [ 72,  70,  62],
        ...,
        [131, 120, 112],
        [131, 121, 111],
        [130, 120, 110]],

       [[ 68,  65,  61],
        [ 72,  69,  65],
        [ 73,  71,  63],
        ...,
        [129, 118, 110],
        [132, 120, 110],
        [130, 118, 108]],

       [[ 74,  70,  65],
        [ 75,  71,  66],
        [ 75,  73,  65],
        ...,
        [125, 116, 106],
        [128, 119, 106],
        [130, 121, 108]]

In [34]:
#-- Localize the object
obj = np.empty((len(good_matches),2), dtype=np.float32)
scene = np.empty((len(good_matches),2), dtype=np.float32)
for i in range(len(good_matches)):
    #-- Get the keypoints from the good matches
    obj[i,0] = kp1[good_matches[i].queryIdx].pt[0]
    obj[i,1] = kp1[good_matches[i].queryIdx].pt[1]
    scene[i,0] = kp2[good_matches[i].trainIdx].pt[0]
    scene[i,1] = kp2[good_matches[i].trainIdx].pt[1]

In [35]:
H, _ =  cv2.findHomography(obj, scene, cv2.RANSAC)
#-- Get the corners from the image_1 ( the object to be "detected" )
obj_corners = np.empty((4,1,2), dtype=np.float32)
obj_corners[0,0,0] = 0
obj_corners[0,0,1] = 0
obj_corners[1,0,0] = img1.shape[1]
obj_corners[1,0,1] = 0
obj_corners[2,0,0] = img1.shape[1]
obj_corners[2,0,1] = img1.shape[0]
obj_corners[3,0,0] = 0
obj_corners[3,0,1] = img1.shape[0]
scene_corners = cv2.perspectiveTransform(obj_corners, H)

In [36]:
#-- Draw lines between the corners (the mapped object in the scene - image_2 )
cv2.line(img_matches, (int(scene_corners[0,0,0] + img1.shape[1]), int(scene_corners[0,0,1])),\
    (int(scene_corners[1,0,0] + img1.shape[1]), int(scene_corners[1,0,1])), (0,255,0), 4)
cv2.line(img_matches, (int(scene_corners[1,0,0] + img1.shape[1]), int(scene_corners[1,0,1])),\
    (int(scene_corners[2,0,0] + img1.shape[1]), int(scene_corners[2,0,1])), (0,255,0), 4)
cv2.line(img_matches, (int(scene_corners[2,0,0] + img1.shape[1]), int(scene_corners[2,0,1])),\
    (int(scene_corners[3,0,0] + img1.shape[1]), int(scene_corners[3,0,1])), (0,255,0), 4)
cv2.line(img_matches, (int(scene_corners[3,0,0] + img1.shape[1]), int(scene_corners[3,0,1])),\
    (int(scene_corners[0,0,0] + img1.shape[1]), int(scene_corners[0,0,1])), (0,255,0), 4)

array([[[ 90,  82,  69],
        [ 92,  83,  70],
        [ 91,  82,  69],
        ...,
        [133, 121, 115],
        [126, 112, 106],
        [129, 115, 109]],

       [[ 93,  83,  73],
        [ 94,  84,  74],
        [ 95,  83,  71],
        ...,
        [137, 125, 121],
        [136, 122, 116],
        [132, 118, 112]],

       [[ 98,  86,  76],
        [ 98,  86,  76],
        [ 94,  85,  72],
        ...,
        [135, 122, 120],
        [134, 122, 116],
        [133, 121, 115]],

       ...,

       [[ 69,  69,  63],
        [ 72,  69,  64],
        [ 72,  70,  62],
        ...,
        [131, 120, 112],
        [131, 121, 111],
        [130, 120, 110]],

       [[ 68,  65,  61],
        [ 72,  69,  65],
        [ 73,  71,  63],
        ...,
        [129, 118, 110],
        [132, 120, 110],
        [130, 118, 108]],

       [[ 74,  70,  65],
        [ 75,  71,  66],
        [ 75,  73,  65],
        ...,
        [125, 116, 106],
        [128, 119, 106],
        [130, 121, 108]]

In [37]:
kerem.show_images(
    [img_matches],
    divided_by=6
)